In [1]:
import scraper as s
import pandas as pd
import numpy as np
import datetime
m = s.stockMongo()

In [5]:
apple = m.get_options("AAPL")

In [16]:
apple.tail()

,ask,bid,contract,in-money,iv,last,last_trade,strike,strike-date,type,volume
date,,,,,,,,,,,
2020-10-29 20:16:26.397,24.90,24.65,AAPL201030P00140000,in-the-money,128.91%,25.22,2020-09-30,140.0,2020-10-30,Put,2.0
2020-10-29 20:16:26.397,29.80,29.50,AAPL201030P00145000,in-the-money,153.52%,34.37,2020-09-11,145.0,2020-10-30,Put,2.0
2020-10-29 20:16:26.397,34.70,34.45,AAPL201030P00150000,in-the-money,137.50%,41.40,2020-09-24,150.0,2020-10-30,Put,1.0
2020-10-29 20:16:26.397,44.65,44.30,AAPL201030P00160000,in-the-money,0.00%,46.81,2020-09-21,160.0,2020-10-30,Put,NaN
2020-10-29 20:16:26.397,49.70,49.25,AAPL201030P00165000,in-the-money,179.69%,53.30,2020-09-23,165.0,2020-10-30,Put,NaN


In [12]:
new_apple = apple.where(apple['last_trade']==apple.index.date).dropna()
new_apple = new_apple[new_apple['volume'] > 50]

In [8]:
apl = m.get_stock_data('AAPL')

In [9]:
def split(apple):
    if apple['date'] < datetime.datetime.fromisoformat('2020-08-31'):
        return apple['c'] * 4
    else:
        return apple['c']

In [13]:
new_apple = new_apple.reset_index()
new_apple['date'] = new_apple['date'].dt.normalize()
new_apple = pd.merge(new_apple, apl[['c', 'date']], on='date')
#apple2 = apple2.set_index('date')
new_apple['c'] = new_apple.apply(split, axis=1)
new_apple.head()

,date,ask,bid,contract,in-money,iv,last,last_trade,strike,strike-date,type,volume,c
0,2020-08-03,138.05,137.25,AAPL200821C00300000,in-the-money,101.69%,136.00,2020-08-03,300.0,2020-08-21,Call,53.0,435.75
1,2020-08-03,117.85,116.85,AAPL200821C00320000,in-the-money,83.81%,117.53,2020-08-03,320.0,2020-08-21,Call,51.0,435.75
2,2020-08-03,102.95,101.85,AAPL200821C00335000,in-the-money,73.96%,103.99,2020-08-03,335.0,2020-08-21,Call,103.0,435.75
3,2020-08-03,97.80,96.95,AAPL200821C00340000,in-the-money,70.34%,98.00,2020-08-03,340.0,2020-08-21,Call,72.0,435.75
4,2020-08-03,93.00,92.10,AAPL200821C00345000,in-the-money,68.53%,90.00,2020-08-03,345.0,2020-08-21,Call,89.0,435.75


In [14]:
new_apple['time'] = new_apple['strike-date'] - new_apple['date']
new_apple['iv'] = pd.Series(new_apple['iv']).str.replace('%', '0', regex=False)
new_apple['iv'] = pd.to_numeric(new_apple['iv'],errors='coerce')
new_apple.head()

,date,ask,bid,contract,in-money,iv,last,last_trade,strike,strike-date,type,volume,c,time
0,2020-08-03,138.05,137.25,AAPL200821C00300000,in-the-money,101.69,136.00,2020-08-03,300.0,2020-08-21,Call,53.0,435.75,18 days
1,2020-08-03,117.85,116.85,AAPL200821C00320000,in-the-money,83.81,117.53,2020-08-03,320.0,2020-08-21,Call,51.0,435.75,18 days
2,2020-08-03,102.95,101.85,AAPL200821C00335000,in-the-money,73.96,103.99,2020-08-03,335.0,2020-08-21,Call,103.0,435.75,18 days
3,2020-08-03,97.80,96.95,AAPL200821C00340000,in-the-money,70.34,98.00,2020-08-03,340.0,2020-08-21,Call,72.0,435.75,18 days
4,2020-08-03,93.00,92.10,AAPL200821C00345000,in-the-money,68.53,90.00,2020-08-03,345.0,2020-08-21,Call,89.0,435.75,18 days


In [15]:
new_apple2 = new_apple
new_apple2 = new_apple2.drop(['volume', 'iv', 'in-money', 'bid', 'ask','last_trade', 'contract'], axis=1)
new_apple = new_apple2.reset_index()
new_apple2 = new_apple2.set_index(['strike-date', 'type'])
new_apple2['date'] = pd.to_datetime(new_apple2['date']).dt.strftime('%m-%d-%Y')
new_apple2['date'] = pd.to_datetime(new_apple2['date'])
strike_pivot = new_apple2.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns
strike_cols

DatetimeIndex(['2020-08-07', '2020-08-14', '2020-08-21', '2020-08-28',
               '2020-09-04', '2020-09-11', '2020-09-18', '2020-09-25',
               '2020-10-02', '2020-10-09', '2020-10-16', '2020-10-23',
               '2020-10-30', '2020-11-06', '2020-11-13', '2020-11-20',
               '2020-11-27', '2020-12-18', '2021-01-15', '2021-06-18',
               '2021-09-17', '2022-01-21', '2022-06-17', '2022-09-16'],
              dtype='datetime64[ns]', name='strike-date', freq=None)

In [ ]:
new_apple3 = []

for k in range(8,10):
    new_apple2 = new_apple2.reset_index()
    new_apple2 = new_apple2.set_index(['type','strike-date'])
    apple_pivot_call = new_apple2.loc[('Call', strike_cols[k])]
    strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
    strike_value_cols = strike_pivot.columns
    apple_pivot_put = new_apple2.loc[('Put', strike_cols[k])]
    strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
    strike_value_puts = strike_pivot.columns

    new_apple2 = new_apple2.reset_index()
    new_apple2 = new_apple2.set_index(['type', 'strike','strike-date'])
    
    

In [4]:
for i in range(len(strike_value_cols)-1):
        
        for j in range(len(strike_value_puts)-1):
            aCLo = apple2.loc[('Call', strike_value_cols[i], strike_cols[k])]
            aCLo = aCLo.reset_index()
            #aCLo = aCLo.drop(['type', 'strike-date'], axis=1)
            aCLo.rename(columns = {'last':'priceCLo', 'strike': 'strikeCLo'}, inplace = True)
            aCHi = apple2.loc[('Call', strike_value_cols[i+1], strike_cols[k])]
            aCHi = aCHi.reset_index()
            #aCHi = aCHi.drop(['type', 'strike-date'], axis=1)
            aCHi.rename(columns = {'last':'priceCHi', 'strike': 'strikeCHi'}, inplace = True)
            aPLo = apple2.loc[('Put', strike_value_puts[j], strike_cols[k])]
            aPLo = aPLo.reset_index()
            #aPLo = aPLo.drop(['type', 'strike-date'], axis=1)
            aPLo.rename(columns = {'last':'pricePLo', 'strike': 'strikePLo'}, inplace = True)
            aPHi = apple2.loc[('Put', strike_value_puts[j+1], strike_cols[k])]
            aPHi = aPHi.reset_index()
            #aPHi = aPHi.drop(['type', 'strike-date'], axis=1)
            aPHi.rename(columns = {'last':'pricePHi', 'strike': 'strikePHi'}, inplace = True)

            a = pd.merge(aCLo, aCHi, on=['date', 'strike-date'])
            a = pd.merge(a, aPLo, on=['date', 'strike-date'])
            a = pd.merge(a, aPHi, on=['date', 'strike-date'])
            a = pd.merge(a, apl[['c', 'date']], on='date')
            a = a.drop(['type_x', 'type_y'], axis=1)
            a['combo'] = "C"+ str(strike_value_cols[i+1]) + "P" + str(strike_value_puts[j])

            new_apple.append(a)

print(new_apple)


OperationFailure: noTimeout cursors are disallowed in this atlas tier